# Operational Trading Pipeline - Chapters 06-07, 22-25

This notebook demonstrates the operational components of an algorithmic trading system,
covering the full pipeline from strategy definition through backtesting, AI-assisted analysis,
risk management, and trade monitoring.

**Chapters covered:**
- **Ch 06-07**: Strategy framework, backtesting engine, walk-forward analysis
- **Ch 22**: AI-assisted trading (LLM sentiment, news signals, portfolio agent)
- **Ch 24**: Risk management (position sizing, stop losses, portfolio risk controls)
- **Ch 25**: Monitoring & analytics (trade logging, P&L tracking, benchmark comparison)

All examples use mock or synthetic data so no external API keys or live connections are required.

## 1. Strategy Framework

Every strategy in Puffin implements the `Strategy` base class with two required methods:
- `generate_signals(data)` -- produces a `SignalFrame` with `signal` (-1 to 1) and `confidence` columns
- `get_parameters()` -- returns a dict of current parameters for reproducibility

The strategy registry provides dynamic discovery via `list_strategies()` and `get_strategy(name)`.

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

from puffin.strategies import Strategy, MomentumStrategy, MeanReversionStrategy, list_strategies

# Inspect available strategies from the registry
print("Registered strategies:", list_strategies())

# Create a momentum strategy with custom parameters
momentum = MomentumStrategy(short_window=10, long_window=30, ma_type="ema")
print(f"\nStrategy: {momentum.name}")
print(f"Parameters: {momentum.get_parameters()}")

# Generate synthetic OHLCV data for demonstration
np.random.seed(42)
n_days = 200
dates = pd.bdate_range(start="2024-01-02", periods=n_days)
price = 100 + np.cumsum(np.random.randn(n_days) * 0.8)

data = pd.DataFrame({
    "Open": price + np.random.randn(n_days) * 0.2,
    "High": price + abs(np.random.randn(n_days) * 0.5),
    "Low": price - abs(np.random.randn(n_days) * 0.5),
    "Close": price,
    "Volume": np.random.randint(1_000_000, 5_000_000, n_days),
}, index=dates)

# Generate signals
signals = momentum.generate_signals(data)
print(f"\nSignal distribution:")
print(signals["signal"].value_counts().sort_index())
print(f"\nFirst 5 signals with confidence:")
print(signals.dropna().head())

## 2. Backtesting Engine

The `Backtester` is an event-driven engine that processes bars sequentially with no lookahead bias.
It supports configurable slippage and commission models, and produces a `BacktestResult` with
equity curve, fills, and performance metrics.

`walk_forward()` implements rolling train/test validation to detect overfitting.

In [ ]:
from puffin.backtest import Backtester, BacktestResult, walk_forward
from puffin.backtest.engine import SlippageModel, CommissionModel

# Configure realistic trading costs
slippage = SlippageModel(fixed=0.0, pct=0.001)      # 10 bps slippage
commission = CommissionModel(flat=1.0, per_share=0.005)  # $1 flat + $0.005/share

bt = Backtester(
    initial_capital=100_000.0,
    slippage=slippage,
    commission=commission,
)

# Run backtest with the momentum strategy on our synthetic data
result = bt.run(momentum, data)

# Display performance metrics
metrics = result.metrics(risk_free_rate=0.05)
print("=== Backtest Performance ===")
for key, value in metrics.items():
    if isinstance(value, float):
        print(f"  {key:25s}: {value:>10.4f}")
    else:
        print(f"  {key:25s}: {value:>10d}")

print(f"\nFills recorded: {len(result.fills)}")
if result.fills:
    print(f"First fill: {result.fills[0].side} {result.fills[0].qty} @ ${result.fills[0].price:.2f}")

In [ ]:
# Walk-forward analysis: detect overfitting by comparing in-sample vs out-of-sample
wf_results = walk_forward(
    strategy=momentum,
    data=data,
    train_ratio=0.7,
    n_splits=4,
    initial_capital=100_000.0,
)

print("=== Walk-Forward Analysis ===")
for split in wf_results:
    train_ret = split["train_metrics"]["total_return"]
    test_ret = split["test_metrics"]["total_return"]
    train_sharpe = split["train_metrics"].get("sharpe_ratio", 0)
    test_sharpe = split["test_metrics"].get("sharpe_ratio", 0)
    print(
        f"  Split {split['split']}: "
        f"Train return={train_ret:+.2%} (Sharpe={train_sharpe:.2f})  |  "
        f"Test return={test_ret:+.2%} (Sharpe={test_sharpe:.2f})"
    )

## 3. AI-Assisted Trading

The AI module provides LLM-powered analysis through a provider abstraction pattern.
Any `LLMProvider` subclass can be swapped in (Claude, OpenAI, local models).

Key components:
- `analyze_sentiment()` -- single-text sentiment scoring
- `NewsSignalGenerator` -- batch news-to-signal pipeline
- `PortfolioAgent` -- AI-driven allocation recommendations

Below we use a mock provider so no API keys are needed.

In [ ]:
import json
from puffin.ai import LLMProvider, analyze_sentiment, NewsSignalGenerator, PortfolioAgent


class MockLLMProvider(LLMProvider):
    """Mock provider that returns deterministic sentiment for testing."""

    def analyze(self, text: str, prompt: str) -> dict:
        # Simple heuristic: count positive vs negative keywords
        positive = sum(1 for w in ["beat", "surge", "record", "growth", "strong", "up"]
                       if w in text.lower())
        negative = sum(1 for w in ["miss", "decline", "loss", "weak", "down", "cut"]
                       if w in text.lower())
        score = min(1.0, max(-1.0, (positive - negative) * 0.4))
        sentiment = "bullish" if score > 0.2 else "bearish" if score < -0.2 else "neutral"

        # Extract ticker-like tokens (all-caps 2-5 chars)
        import re
        tickers = re.findall(r'\b[A-Z]{2,5}\b', text)
        tickers = [t for t in tickers if t not in {"CEO", "NYSE", "CFO", "EPS", "IPO", "GDP"}]

        response = json.dumps({
            "sentiment": sentiment,
            "score": score,
            "confidence": 0.7 + abs(score) * 0.2,
            "reasoning": f"{'Positive' if score > 0 else 'Negative' if score < 0 else 'Neutral'} signals detected.",
            "tickers_mentioned": tickers,
        })
        return {"response": response, "model": "mock-v1", "usage": {"tokens": len(text.split())}}

    def generate(self, prompt: str, system: str | None = None) -> str:
        return json.dumps({
            "recommendations": [
                {"ticker": "AAPL", "action": "hold", "target_allocation_pct": 25.0,
                 "reasoning": "Strong fundamentals with stable growth."},
                {"ticker": "TSLA", "action": "reduce", "target_allocation_pct": 10.0,
                 "reasoning": "Elevated volatility warrants position reduction."},
            ],
            "market_outlook": "neutral",
            "risk_assessment": "medium",
            "summary": "Mixed signals suggest maintaining a balanced approach.",
        })


provider = MockLLMProvider(cache_dir="/tmp/puffin_mock_cache")

# --- Single-text sentiment analysis ---
article = "AAPL beat earnings expectations with record revenue growth. CEO highlighted strong demand."
sentiment = analyze_sentiment(provider, article)
print("=== Sentiment Analysis ===")
for k, v in sentiment.items():
    print(f"  {k}: {v}")

# --- Batch news signal generation ---
articles = [
    {"text": "AAPL beat earnings expectations with record revenue.", "timestamp": "2024-10-28T09:00:00"},
    {"text": "TSLA missed delivery targets, stock down in premarket.", "timestamp": "2024-10-28T08:30:00"},
    {"text": "AAPL strong growth in services segment drives surge.", "timestamp": "2024-10-28T10:00:00"},
    {"text": "MSFT reports record cloud revenue, beat on all metrics.", "timestamp": "2024-10-28T16:05:00"},
]

signal_gen = NewsSignalGenerator(
    provider=provider,
    bullish_threshold=0.3,
    bearish_threshold=-0.3,
    min_confidence=0.5,
    min_articles=1,
)
news_signals = signal_gen.generate(articles)
print("\n=== News Signals ===")
print(news_signals)

In [ ]:
# --- Portfolio Agent: AI-driven allocation recommendations ---
agent = PortfolioAgent(provider=provider)

recommendations = agent.recommend(
    positions={
        "AAPL": {"qty": 200, "avg_price": 170.00, "current_price": 185.50},
        "TSLA": {"qty": 50, "avg_price": 240.00, "current_price": 215.00},
    },
    market_data={
        "AAPL": {"price": 185.50, "change_pct": 2.1, "volume": 45_000_000},
        "TSLA": {"price": 215.00, "change_pct": -3.5, "volume": 80_000_000},
        "MSFT": {"price": 420.00, "change_pct": 1.8, "volume": 22_000_000},
    },
    sentiment={"AAPL": {"score": 0.6, "confidence": 0.8}, "TSLA": {"score": -0.4, "confidence": 0.7}},
    strategy_signals={"AAPL": 0.8, "TSLA": -0.3, "MSFT": 0.5},
)

print("=== Portfolio Agent Recommendations ===")
print(f"Market outlook: {recommendations['market_outlook']}")
print(f"Risk assessment: {recommendations['risk_assessment']}")
print(f"Summary: {recommendations['summary']}")
print("\nRecommendations:")
for rec in recommendations.get("recommendations", []):
    print(f"  {rec['ticker']:6s} -> {rec['action']:8s} (target: {rec['target_allocation_pct']}%)")
    print(f"           {rec['reasoning']}")

print(f"\nDecision log entries: {len(agent.get_decision_log())}")

## 4. Risk Management

The risk module provides three layers of protection:

1. **Position Sizing** -- `fixed_fractional`, `kelly_criterion`, `volatility_based` control how much capital to allocate per trade
2. **Stop Losses** -- `FixedStop`, `TrailingStop`, `ATRStop`, `TimeStop` managed by `StopLossManager`
3. **Portfolio Risk** -- `PortfolioRiskManager` with drawdown checks, exposure limits, circuit breaker, VaR/CVaR

In [ ]:
from puffin.risk import (
    fixed_fractional, kelly_criterion, volatility_based,
    FixedStop, TrailingStop, ATRStop, StopLossManager,
    PortfolioRiskManager,
)
from puffin.risk.stop_loss import Position as StopPosition

# === Position Sizing ===
equity = 100_000.0

# Fixed fractional: risk 2% of equity with $5 stop distance
size_ff = fixed_fractional(equity=equity, risk_pct=0.02, stop_distance=5.0)
print("=== Position Sizing ===")
print(f"Fixed fractional (2% risk, $5 stop): {size_ff:.0f} shares")

# Kelly criterion: 55% win rate, 1.5:1 win/loss ratio, half Kelly
kelly_pct = kelly_criterion(win_rate=0.55, win_loss_ratio=1.5, fraction=0.5)
print(f"Half Kelly allocation: {kelly_pct:.2%} of equity (${equity * kelly_pct:,.0f})")

# Volatility-based: ATR of $3, 2x multiplier
size_vol = volatility_based(equity=equity, atr=3.0, risk_pct=0.02, multiplier=2.0)
print(f"Volatility-based (ATR=$3, 2x): {size_vol:.0f} shares")

# === Stop Loss Manager ===
print("\n=== Stop Loss Manager ===")
slm = StopLossManager()

# Create a long position in AAPL
pos = StopPosition(
    ticker="AAPL",
    entry_price=185.00,
    entry_time=datetime(2024, 10, 28, 9, 30),
    quantity=200,
    side="long",
    metadata={"atr": 3.5},
)
slm.add_position(pos)

# Layer multiple stop types
slm.add_stop("AAPL", FixedStop(stop_distance=10.0))       # $10 fixed stop
slm.add_stop("AAPL", TrailingStop(trail_distance=0.05, price_based=False))  # 5% trailing
slm.add_stop("AAPL", ATRStop(atr_multiplier=2.0))          # 2x ATR stop

# Check stop prices
stop_prices = slm.get_stop_prices("AAPL", current_price=188.00)
print(f"Position: AAPL long @ $185.00")
print(f"Current stop levels at price=$188.00:")
for name, price in stop_prices.items():
    print(f"  {name}: ${price:.2f}")

# Simulate price drop -- check which stops trigger
test_prices = [188.0, 183.0, 178.0, 174.0]
print(f"\nStop trigger check:")
for p in test_prices:
    triggered = slm.check_stops("AAPL", p)
    print(f"  Price=${p:.2f} -> {'TRIGGERED' if triggered else 'OK'}")

In [ ]:
from puffin.risk.portfolio_risk import Position as RiskPosition

# === Portfolio-Level Risk Controls ===
prm = PortfolioRiskManager()

# Simulate an equity curve with a drawdown
equity_values = [100_000, 102_000, 105_000, 103_000, 98_000, 95_000, 97_000, 99_000]
equity_curve = pd.Series(equity_values, index=pd.bdate_range("2024-10-21", periods=8))

# Drawdown check
ok, current_dd = prm.check_drawdown(equity_curve, max_dd=0.10)
print("=== Portfolio Risk Controls ===")
print(f"Drawdown check (10% limit): {'PASS' if ok else 'FAIL'}, current={current_dd:.2%}")

# Circuit breaker
halted = prm.circuit_breaker(equity_curve, threshold=0.12)
print(f"Circuit breaker (12% threshold): {'HALTED' if halted else 'OK'}")

# Exposure check
positions = [
    RiskPosition("AAPL", 200, 185.0, 37_000, 0.40),
    RiskPosition("TSLA", 50, 215.0, 10_750, 0.12),
    RiskPosition("MSFT", 100, 420.0, 42_000, 0.45),
]
ok_exp, exposure = prm.check_exposure(positions, max_exposure=1.0)
print(f"Exposure check (100% limit): {'PASS' if ok_exp else 'FAIL'}, current={exposure:.2%}")

# Concentration metrics
conc = prm.concentration_metrics(positions)
print(f"\nConcentration metrics:")
for k, v in conc.items():
    print(f"  {k}: {v:.4f}" if isinstance(v, float) else f"  {k}: {v}")

# VaR and Expected Shortfall
returns = equity_curve.pct_change().dropna()
var_95 = prm.compute_var(returns, confidence=0.95)
es_95 = prm.compute_expected_shortfall(returns, confidence=0.95)
print(f"\nValue at Risk (95%): {var_95:.4f}")
print(f"Expected Shortfall (95%): {es_95:.4f}")

## 5. Trade Monitoring & P&L Tracking

The monitoring module provides:
- `TradeLog` / `TradeRecord` -- immutable audit trail of every execution
- `PnLTracker` -- real-time P&L with realized/unrealized breakdown and strategy attribution

In [ ]:
from puffin.monitor import TradeLog, TradeRecord, PnLTracker

# === Trade Logging ===
log = TradeLog()

# Record a series of trades
trades = [
    TradeRecord(datetime(2024, 10, 28, 9, 31), "AAPL", "buy",  200, 184.50, 2.00, 0.18, "momentum"),
    TradeRecord(datetime(2024, 10, 28, 9, 35), "TSLA", "buy",   50, 214.80, 1.50, 0.21, "momentum"),
    TradeRecord(datetime(2024, 10, 29, 10, 15), "MSFT", "buy", 100, 419.20, 1.80, 0.42, "mean_reversion"),
    TradeRecord(datetime(2024, 10, 30, 14, 45), "AAPL", "sell", 200, 189.30, 2.00, 0.19, "momentum"),
    TradeRecord(datetime(2024, 10, 31, 11, 00), "TSLA", "sell",  50, 210.50, 1.50, 0.21, "momentum"),
]

for t in trades:
    log.record(t)

# Trade log summary
summary = log.summary()
print("=== Trade Log Summary ===")
for k, v in summary.items():
    print(f"  {k}: {v}")

# Filter by strategy
momentum_trades = log.filter(strategy="momentum")
print(f"\nMomentum strategy trades: {len(momentum_trades)}")

# Filter by ticker
aapl_trades = log.filter(ticker="AAPL")
print(f"AAPL trades: {len(aapl_trades)}")

In [ ]:
# === P&L Tracking ===
from puffin.monitor.pnl import Position as PnLPosition

tracker = PnLTracker(initial_cash=100_000.0)

# Replay trades through the P&L tracker
tracker.record_trade("AAPL", 200, 184.50, "buy", commission=2.00)
tracker.record_trade("TSLA", 50, 214.80, "buy", commission=1.50)

# Update with current market prices
tracker.update(tracker.positions, {"AAPL": 186.00, "TSLA": 212.00})

# Record sells
tracker.record_trade("AAPL", 200, 189.30, "sell", commission=2.00)
tracker.update(tracker.positions, {"TSLA": 210.50})

tracker.record_trade("TSLA", 50, 210.50, "sell", commission=1.50)
tracker.update(tracker.positions, {})

# Performance summary
perf = tracker.performance_summary()
print("=== P&L Performance Summary ===")
for k, v in perf.items():
    if isinstance(v, float):
        print(f"  {k:25s}: {v:>12.2f}")
    else:
        print(f"  {k:25s}: {v}")

print(f"\nHistory snapshots recorded: {len(tracker.history)}")

## 6. Benchmark Comparison

`BenchmarkComparison` computes alpha, beta, information ratio, and tracking error
to measure strategy performance against a market benchmark.

In [ ]:
from puffin.monitor import BenchmarkComparison

# Generate synthetic strategy and benchmark returns
np.random.seed(123)
n_periods = 252
dates = pd.bdate_range("2024-01-02", periods=n_periods)

# Benchmark: market with slight upward drift
benchmark_returns = pd.Series(
    np.random.randn(n_periods) * 0.01 + 0.0003,
    index=dates,
    name="benchmark",
)

# Strategy: correlated with benchmark but with extra alpha
strategy_returns = pd.Series(
    benchmark_returns * 0.8 + np.random.randn(n_periods) * 0.005 + 0.0002,
    index=dates,
    name="strategy",
)

bc = BenchmarkComparison()
metrics = bc.compare(strategy_returns, benchmark_returns)

print("=== Benchmark Comparison ===")
for k, v in metrics.items():
    print(f"  {k:20s}: {v:>10.4f}")

# Cumulative return comparison
cum_strategy = (1 + strategy_returns).cumprod()
cum_benchmark = (1 + benchmark_returns).cumprod()
print(f"\nStrategy cumulative return: {(cum_strategy.iloc[-1] - 1):.2%}")
print(f"Benchmark cumulative return: {(cum_benchmark.iloc[-1] - 1):.2%}")

## 7. Full Pipeline: Strategy -> Backtest -> Risk -> Monitor

This section combines all operational components into a single end-to-end pipeline:

1. Define strategy and run backtest
2. Compute position sizes using risk management
3. Check portfolio-level risk constraints
4. Log trades and track P&L
5. Compare against benchmark

In [ ]:
# --- Full Operational Pipeline ---

# Step 1: Strategy + Backtest
strategy = MomentumStrategy(short_window=15, long_window=40, ma_type="ema")

np.random.seed(99)
n_days = 252
dates = pd.bdate_range("2024-01-02", periods=n_days)
price = 150 + np.cumsum(np.random.randn(n_days) * 1.2)
pipeline_data = pd.DataFrame({
    "Open": price + np.random.randn(n_days) * 0.3,
    "High": price + abs(np.random.randn(n_days) * 0.6),
    "Low": price - abs(np.random.randn(n_days) * 0.6),
    "Close": price,
    "Volume": np.random.randint(500_000, 3_000_000, n_days),
}, index=dates)

bt = Backtester(
    initial_capital=100_000.0,
    slippage=SlippageModel(pct=0.001),
    commission=CommissionModel(flat=1.0, per_share=0.005),
)
result = bt.run(strategy, pipeline_data)

bt_metrics = result.metrics()
print("Step 1: Backtest Complete")
print(f"  Total return: {bt_metrics['total_return']:+.2%}")
print(f"  Sharpe ratio: {bt_metrics['sharpe_ratio']:.2f}")
print(f"  Max drawdown: {bt_metrics['max_drawdown']:.2%}")
print(f"  Total trades: {bt_metrics['total_trades']}")

# Step 2: Position sizing for next trade
current_equity = result.final_value
atr_estimate = pipeline_data["Close"].diff().abs().rolling(14).mean().iloc[-1]

size = fixed_fractional(equity=current_equity, risk_pct=0.02, stop_distance=atr_estimate * 2)
kelly_alloc = kelly_criterion(
    win_rate=bt_metrics.get("win_rate", 0.5),
    win_loss_ratio=max(bt_metrics.get("profit_factor", 1.0), 0.01),
    fraction=0.25,  # quarter Kelly for safety
)

print(f"\nStep 2: Position Sizing")
print(f"  Fixed fractional (2% risk): {size:.0f} shares")
print(f"  Quarter Kelly allocation: {kelly_alloc:.2%} of ${current_equity:,.0f} = ${current_equity * kelly_alloc:,.0f}")

# Step 3: Portfolio risk check
prm = PortfolioRiskManager()
ok_dd, dd = prm.check_drawdown(result.equity_curve, max_dd=0.15)
halted = prm.circuit_breaker(result.equity_curve, threshold=0.20)

print(f"\nStep 3: Risk Controls")
print(f"  Drawdown check (15% limit): {'PASS' if ok_dd else 'FAIL'} (current: {dd:.2%})")
print(f"  Circuit breaker (20%): {'HALTED' if halted else 'OK'}")

# Step 4: Log trades and track P&L
pipeline_log = TradeLog()
pipeline_tracker = PnLTracker(initial_cash=100_000.0)

for fill in result.fills[:6]:  # Log first 6 fills as example
    record = TradeRecord(
        timestamp=fill.timestamp if isinstance(fill.timestamp, datetime) else datetime.now(),
        ticker="SYN",
        side=fill.side,
        qty=fill.qty,
        price=fill.price,
        commission=fill.commission,
        slippage=fill.slippage,
        strategy="momentum_ema",
    )
    pipeline_log.record(record)

log_summary = pipeline_log.summary()
print(f"\nStep 4: Trade Monitoring")
print(f"  Trades logged: {log_summary['total_trades']}")
print(f"  Buy/Sell: {log_summary['total_buy']}/{log_summary['total_sell']}")
print(f"  Total commission: ${log_summary['total_commission']:.2f}")
print(f"  Avg trade size: ${log_summary['avg_trade_size']:,.2f}")

# Step 5: Benchmark comparison
strategy_returns = result.equity_curve.pct_change().dropna()
benchmark_returns = pd.Series(
    np.random.randn(len(strategy_returns)) * 0.008 + 0.0003,
    index=strategy_returns.index,
)

bc = BenchmarkComparison()
bm_metrics = bc.compare(strategy_returns, benchmark_returns)

print(f"\nStep 5: Benchmark Comparison")
print(f"  Alpha: {bm_metrics['alpha']:.6f}")
print(f"  Beta: {bm_metrics['beta']:.4f}")
print(f"  Information ratio: {bm_metrics['ir']:.4f}")
print(f"  Tracking error: {bm_metrics['tracking_error']:.4f}")

print("\n=== Pipeline Complete ===")